In [1]:
#Ejercicio 3
#Escribe un trozo de código que sirva para definir un modelo en Keras que combine al
#menos una capa LSTM, tres convolucionales y una densa. Muestra el resumen del modelo por
#pantalla.
#Tenemos 10000 datos de series temporales de 5 activos financieros de longitud 4, i.e.
#(10000,4,5). Las salidas son las predicciones para el instante temporal siguiente, i.e. (10000,1).

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, LSTM, Dropout
import numpy as np
import pandas as pd

In [3]:
#AQUI TENGO QUE PONER LOS DATOS DE LOS 5 ACTIVOS DE LONGITUD 4
#hist_google = pd.read_csv("C:\\Users\\ignac\\OneDrive\\Escritorio\\BME\\Master\\CLASES\\Modulo 4\\Redes Convolucionales\\Diccionario\\Leer_datos\\googl.us.txt", delimiter=",")
hist_google = pd.read_csv("googl.us.txt", delimiter=",")
print(hist_google)

            Date      Open     High       Low     Close    Volume  OpenInt
0     2004-08-19    50.000    52.03    47.980    50.170  44703800        0
1     2004-08-20    50.505    54.54    50.250    54.155  22857200        0
2     2004-08-23    55.375    56.74    54.525    54.700  18274400        0
3     2004-08-24    55.620    55.80    51.785    52.435  15262600        0
4     2004-08-25    52.480    54.00    51.940    53.000   9197800        0
...          ...       ...      ...       ...       ...       ...      ...
3328  2017-11-06  1049.100  1052.59  1042.000  1042.680    913954        0
3329  2017-11-07  1049.650  1053.41  1043.000  1052.390   1303832        0
3330  2017-11-08  1050.050  1062.69  1047.050  1058.290   1214469        0
3331  2017-11-09  1048.000  1050.88  1035.850  1047.720   1793994        0
3332  2017-11-10  1043.870  1046.63  1041.220  1044.150    970498        0

[3333 rows x 7 columns]


In [4]:
#PRUEBA PARA VER SI PUEDO TRABAJAR CON 10.000 DATOS
from api_bme import APIBMEHandler
APIBME = APIBMEHandler('IBEX', 'ignaciobermejo_algo1')
maestro_df = APIBME.get_ticker_master()
data_close, data_high, data_low, data_open, data_vol = APIBME.get_data()
benchmark = APIBME.get_close_data_ticker('benchmark')

data_close.shape

(2947, 64)

In [5]:
#Divido los datos para para hacer una prueba y ver si funciona con 5 requisitos distintos   que luego habra que cambiar por los activos necesarios
google_op=hist_google['Open']
google_cl=hist_google['Close']
google_vol=hist_google['Volume']
google_high=hist_google['High']
google_low=hist_google['Low']

hist_google.shape

(3333, 7)

In [6]:
#Realizo la particion del modelo en train y test
#forma 10000,4,5
op=google_op.values[np.arange(0,2000,1)]
cl=google_cl.values[np.arange(0,2000,1)]
vol=google_vol.values[np.arange(0,2000,1)]
high=google_high.values[np.arange(0,2000,1)]
low=google_low.values[np.arange(0,2000,1)]

op_train=np.reshape(op,(200,10,1))
cl_train = cl[np.arange(9,2000,10)]

op_train.shape

(200, 10, 1)

In [9]:
#Defino el modelo
model_3 = Sequential()

model_3.add(LSTM(20, input_shape=(4,5), return_sequences=True))
model_3.add(Dropout(0.2))
model_3.add(Conv1D(15, kernel_size=3, padding='same'))
model_3.add(Dropout(0.2))
model_3.add(Conv1D(10, kernel_size=2))
model_3.add(Dropout(0.2))
model_3.add(Conv1D(5,kernel_size=1))
model_3.add(Dropout(0.2))
model_3.add(Dense(1))

In [10]:
#Compilo el modelo
model_3.compile(optimizer='adam',loss='mae')

In [11]:
model_3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 4, 20)             2080      
_________________________________________________________________
dropout (Dropout)            (None, 4, 20)             0         
_________________________________________________________________
conv1d (Conv1D)              (None, 4, 15)             915       
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 15)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 3, 10)             310       
_________________________________________________________________
dropout_2 (Dropout)          (None, 3, 10)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 3, 5)             